In [2]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
oil=pd.read_csv('oil.csv')
stores=pd.read_csv('stores.csv')
holiday=pd.read_csv('holidays_events.csv')

In [50]:
#df.date=pd.to_datetime(df.date,format='%Y-%m-%d')
#oil.date=pd.to_datetime(oil.date,format='%Y-%m-%d')

In [13]:
df=pd.merge(df,oil,how='left')
df_test=pd.merge(df_test,oil,how='left')

In [71]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN


In [14]:
le=LabelEncoder()
df['familynum']=le.fit_transform(df.family)
df.fillna(0,inplace=True)
df_test['familynum']=le.fit_transform(df_test.family)
df_test.fillna(0,inplace=True)

In [15]:
train=df[['store_nbr','onpromotion','dcoilwtico','familynum','sales']]
test=df_test[['store_nbr','onpromotion','dcoilwtico','familynum']]

In [7]:
x=train.drop('sales',axis=1)
y=train[['sales']]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
#print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)#

In [96]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_test, y_test)

params = {
    "objective" : "reg:squarederror",
    "eval_metric" : "rmse"
}
models = xgb.train(
    params = params,
    dtrain = dtrain,
    evals = [(dtrain, "train"), (dvalid, "valid")],
    num_boost_round = 1000
)

[0]	train-rmse:935.01257	valid-rmse:928.57147
[1]	train-rmse:787.77362	valid-rmse:783.87695
[2]	train-rmse:701.48688	valid-rmse:699.34460
[3]	train-rmse:651.06616	valid-rmse:650.35107
[4]	train-rmse:609.37628	valid-rmse:610.59271
[5]	train-rmse:583.77350	valid-rmse:586.62384
[6]	train-rmse:565.38739	valid-rmse:569.02484
[7]	train-rmse:551.15002	valid-rmse:555.48023
[8]	train-rmse:535.20526	valid-rmse:540.10004
[9]	train-rmse:524.32886	valid-rmse:529.62366
[10]	train-rmse:516.74554	valid-rmse:522.36877
[11]	train-rmse:506.24048	valid-rmse:512.55090
[12]	train-rmse:501.03644	valid-rmse:507.25003
[13]	train-rmse:493.67783	valid-rmse:500.22778
[14]	train-rmse:485.71695	valid-rmse:492.51782
[15]	train-rmse:479.56223	valid-rmse:486.53976
[16]	train-rmse:475.48679	valid-rmse:482.67386
[17]	train-rmse:468.50998	valid-rmse:476.47797
[18]	train-rmse:465.05280	valid-rmse:473.12631
[19]	train-rmse:460.63910	valid-rmse:468.93399
[20]	train-rmse:458.57144	valid-rmse:466.99683
[21]	train-rmse:455.966

In [112]:
from sklearn.svm import SVR
lg=SVR(kernel='linear')

lg.fit(x_train,y_train)
lg.predict(x_test)
lg.score(x_test,y_test)


C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
from sklearn.ensemble import RandomForestRegressor
rg=RandomForestRegressor(random_state=0)
rg.fit(x_train,y_train)
sc_tr=rg.score(x_test,y_test)
sc_te=rg.score(x_test,y_test)

C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
sc_tr,sc_te

(0.8784457795520263, 0.8784457795520263)

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor(random_state=0)
gb.fit(x_train,y_train)
sc_trg=gb.score(x_test,y_test)
sc_teg=gb.score(x_test,y_test)


C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
sc_trg,sc_teg

(0.6935556345392584, 0.6935556345392584)

In [17]:
test['sales']=rg.predict(test)

C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_test['sales']=test['sales']

In [24]:
df_test[['id','sales']].to_csv('submission.csv',index=False)